In [ ]:
# Torch Imports
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
# Data Loader Class
import os
import torch
import pandas as pd
import torchvision
import torchvision.io
from torchvision.io.image import read_image
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
import cv2 as io

class Ham10000(Dataset):
    def __init__(self, csv_file, directory, transform, datasetname):
        self.annotations = pd.read_csv(f"{csv_file}")
        self.img_root_dir = 'dataverse_files'
        self.transform = transform
        self.datasetname = datasetname
        self.csv_file = csv_file
        self.directory = directory
        
    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_path = ''
        img_path = os.path.join(self.directory, self.datasetname, self.annotations.iloc[idx, 1])
        img_path += ".jpg"

        x_img = io.imread(img_path)
        if self.transform:
            x_img = self.transform(x_img)

        y_label = torch.tensor(int(self.annotations.iloc[idx, 7]))

        return (x_img, y_label)




In [ ]:
# KFold Function
from sklearn.model_selection import KFold
import torch.nn as nn
import copy

# pass in model constructor
def kfold(model, dataset, device, num_folds=5, num_epochs=10, loss_function=nn.CrossEntropyLoss()):

  results = {}
  kfold = KFold(n_splits=num_folds, shuffle=True)

  print('--------------------------------')
  # K-fold Cross Validation model evaluation
  for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
                      dataset, 
                      batch_size=32, sampler=train_subsampler, num_workers=4)
    testloader = torch.utils.data.DataLoader(
                      dataset,
                      batch_size=32, sampler=test_subsampler, num_workers=4)
    
    # Init the neural network
    network = copy.deepcopy(model)
    network.to(device)
    
    # Initialize optimizer
    optimizer = torch.optim.Adam(network.parameters(), lr=0.001)
    total_train_loss = []
    fold_train_acc = []
    fold_test_acc = []
    for epoch in range(0, num_epochs):
      print(f'Starting epoch {epoch+1}', '-', num_epochs)
      correct, total = 0, 0
      current_loss = 0.0
      network.train()
      train_loss = 0
      for i, data in enumerate(trainloader, 0):
        inputs, targets = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()
        outputs = network(inputs)
        
        _, predicted = torch.max(outputs.data, 1)
        
        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()
        
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
        
        # Print statistics
        train_loss+= loss.item()
        current_loss += loss.item()
      fold_train_acc.append(100.0 * correct / total)

      network.eval()
    # Evaluation for this fold
      correct, total = 0, 0
      total_labels, total_preds = [],[]
      with torch.no_grad():
        for i, data in enumerate(testloader, 0):
          inputs, targets = data[0].to(device), data[1].to(device)
          outputs = network(inputs)
          _, predicted = torch.max(outputs.data, 1)
          total_labels.extend(targets.data.cpu().detach().numpy())
          total_preds.extend(predicted.cpu().detach().numpy())
          total += targets.size(0)
          correct += (predicted == targets).sum().item()
   
        results[fold] = 100.0 * (correct / total)

      total_train_loss.append(train_loss)
      fold_test_acc.append(100.0 * correct / total)
    torch.save(network, "./vanilla_cnn_aug_{0}.pth".format(fold))

    print("fold_train_acc: ", fold_train_acc)
    print("fold_test_acc: ", fold_test_acc)
    print("total_train_loss: ", total_train_loss)
  print(f'K-FOLD CROSS VALIDATION RESULTS FOR {num_folds} FOLDS')
  print('--------------------------------')
  sum = 0.0
  for key, value in results.items():
    print(f'Fold {key}: {value} %')
    sum += value
  print(f'Average: {sum/len(results.items())} %')
  

In [ ]:
# CNN Class Defintion
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 16, 3),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(16, 32, 3),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3),
            nn.MaxPool2d(2, 2),
 
            nn.Conv2d(32, 64, 3),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3),
            nn.MaxPool2d(2, 2),

            nn.Flatten(),
            nn.Linear(128 * 52 * 71, 64),
            nn.ReLU(),
            nn.Linear(64, 7)
        )

    def forward(self, x):
        return self.network(x)
      
    def save(self,path):
      torch.save(self.network.state_dict(), path)
    
    def load(self, path):
      self.network.load_state_dict(torch.load(path))

    def train_net(self,epochs, lr, train_loader, test_loader,opt_func, loss_func):
      print(self.device)
      self.network.train()

      optimizer = opt_func(self.network.parameters(), lr=lr)

      for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
          inputs, labels = data
          inputs, labels = inputs.to(self.device),  labels.to(self.device) # move to GPU is possible
          
          # zero the parameter gradients
          optimizer.zero_grad()

          # forward + backward + optimize
          outputs = self.network(inputs)
          loss = loss_func(outputs, labels)
          loss.backward()
          optimizer.step()

          # print statistics
          running_loss += loss.item()
          if i % 2000 == 1999:    # print every 2000 mini-batches
              print(f'[{epoch + 1}, {i + 1:5d}] train loss: {running_loss / 2000:.3f}')
              running_loss = 0.0

      print('Finished Training')
    
    @torch.no_grad()
    def test(self,test_loader):
      self.network.eval()
      correct = 0
      total = 0
      for data in test_loader:
        inputs, labels = data
        inputs, labels = inputs.to(self.device),  labels.to(self.device) 

        predictions = self.network(inputs)
        _, predicted = torch.max(predictions.data, 1)

        total += len(data)
        correct += torch.sum(predicted == labels).item()
      
      return correct/total

In [ ]:
# Run K-Folds (Vanilla CNN w/o Augmentations)
csv_file = "./dataverse_files/HAM10000_metadata.csv"
directory = "./dataverse_files"

transform = transforms.Compose(
      [
        transforms.ToTensor(),
        transforms.Normalize((0.1411, 0.0923, 0.5270), (0.3407, 0.3058, 0.2824))
      ]
    )

batch_size = 32
datasetname = "HAM10000_images_off"
dataset = Ham10000(csv_file, directory, transform, datasetname)

model = Net()


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

kfold(model, dataset, device)

In [ ]:
# Process Results

fold1trainacc = [66.13829256115827, 66.74987518721917, 67.38642036944583, 67.44882675986021, 67.37393909136296, 66.88716924613081, 66.97453819271094, 66.60009985022467, 66.91213180229656, 66.91213180229656]
fold1testacc = [66.00099850224663, 68.09785322016974, 67.3989016475287, 68.04792810783825, 67.04942586120819, 67.2491263105342, 66.99950074887668, 67.09935097353969, 67.09935097353969, 67.09935097353969]
total1_train_loss =  [538.7118993103504, 483.97415643930435, 451.3105698078871, 438.3517017364502, 480.92257146537304, 469.55596220493317, 494.6970077455044, 885.2272503376007, 602.0155299901962, 577.3336101174355]

fold2trainacc = [66.0134797803295, 66.2755866200699, 66.71243135297054, 66.72491263105341, 66.72491263105341, 66.72491263105341, 66.72491263105341, 66.72491263105341, 66.72491263105341, 66.72491263105341]
fold2testacc = [67.84822765851223, 67.84822765851223, 67.84822765851223, 67.84822765851223, 67.84822765851223, 67.84822765851223, 67.84822765851223, 67.84822765851223, 67.84822765851223, 67.84822765851223]
total2_train_loss=  [535.1830442249775, 519.0360944867134, 579.314981341362, 577.0459206104279, 577.2824673652649, 574.9433191120625, 574.8410721719265, 574.2682507038116, 573.5881962776184, 574.595221221447]

fold3trainacc = [66.2256615077384, 67.41138292561158, 67.22416375436845, 67.51123315027459, 67.32401397903145, 67.1492760858712, 67.16175736395407, 67.41138292561158, 67.83574638042936, 68.24762855716425]
fold3testacc = [65.15227159261109, 65.5017473789316, 65.35197204193709, 65.3020469296056, 65.35197204193709, 65.35197204193709, 65.3020469296056, 65.35197204193709, 66.99950074887668, 67.2491263105342]
total3_train_loss =  [525.2064574062824, 459.3988243639469, 469.3730876892805, 450.1371608674526, 470.49838972091675, 500.6157111674547, 466.9559631049633, 444.4675491452217, 436.5911014974117, 425.83565333485603]

fold4trainacc = [65.80129805292061, 67.44882675986021, 67.83574638042936, 68.26010983524714, 67.68597104343485, 68.02296555167248, 68.47229156265601, 69.9700449326011, 71.80479281078382, 73.52720918622067]
fold4testacc = [66.94957563654518, 68.69695456814777, 68.59710434348477, 69.0464303544683, 69.34598102845732, 68.19770344483275, 69.99500748876684, 69.29605591612581, 69.24613080379432, 69.0464303544683]
total4_train_loss=  [517.8645753860474, 467.08774450421333, 448.84585615992546, 435.18410378694534, 441.71014964580536, 437.12786234915257, 426.4030050933361, 400.3480721116066, 370.62979401648045, 369.1445248275995]

fold5trainacc = [65.95107338991512, 66.74987518721917, 66.58761857214179, 65.05242136794807, 66.71243135297054, 66.7873190214678, 66.7873190214678, 66.7873190214678, 66.7873190214678, 66.7873190214678]
fold5testacc = [68.74687968047928, 67.59860209685472, 67.64852720918623, 67.59860209685472, 67.59860209685472, 67.59860209685472, 67.59860209685472, 67.59860209685472, 67.59860209685472, 67.59860209685472]
total5_train_loss=  [517.5363972485065, 507.1936755105853, 498.6692821085453, 573.3577049970627, 702.2055148482323, 604.909136891365, 581.6331693530083, 574.5571982860565, 572.0612896680832, 571.0651732683182]


total_train_loss_append = [total1_train_loss, total2_train_loss, total3_train_loss, total4_train_loss, total5_train_loss]
total_train_loss = []
for i in range(len(total1_train_loss)):
  total_loss = []
  for j in range(len(total_train_loss_append)):
    total_loss.append(total_train_loss_append[j][i])
  total_train_loss.append(sum(total_loss) / len(total_loss))
print(total_train_loss)


total_train_acc_append = [fold1trainacc, fold2trainacc, fold3trainacc, fold4trainacc, fold5trainacc]

total_train_acc = []
for i in range(len(total1_train_loss)):
  total_acc = []
  for j in range(len(total_train_acc_append)):
    total_acc.append(total_train_acc_append[j][i])
  total_train_acc.append(sum(total_acc) / len(total_acc))
print(total_train_acc)


total_test_acc_append = [fold1testacc, fold2testacc, fold3testacc, fold4testacc, fold5testacc]

total_test_acc = []
for i in range(len(total1_train_loss)):
  total_acc = []
  for j in range(len(total_test_acc_append)):
    total_acc.append(total_test_acc_append[j][i])
  total_test_acc.append(sum(total_acc) / len(total_acc))
print(total_test_acc)

In [ ]:
# Loss Graph
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import seaborn as sns
import pandas as pd

epochs = list(range(0,len(total_train_loss)))

data = pd.DataFrame({
    'Epoch': epochs,
    'Loss': total_train_loss,
})

sns.set_theme(style="darkgrid")
sns.lineplot(x='Epoch',y='Loss',data=data).set(title="Train Loss")

In [ ]:
# Accuracy Graph
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

epochs = list(range(0,len(total_train_acc)))

data = pd.DataFrame({
    'Epoch': epochs,
    'Training Accuracy': total_train_acc,
    'Testing Accuracy': total_test_acc
})

sns.set_theme(style="darkgrid")
sns.lineplot(x='Epoch',y='Accuracy',hue='variable',data=pd.melt(data, ['Epoch'],value_name="Accuracy")).set(title="Train vs Test Accuracy")


In [ ]:
# Vanilla CNN w/Updated Layer Sizes (for Crop Augmentation)

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 16, 3),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(16, 32, 3),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3),
            nn.MaxPool2d(2, 2),
 
            nn.Conv2d(32, 64, 3),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3),
            nn.MaxPool2d(2, 2),

            nn.Flatten(),
            nn.Linear(128 * 24 * 24, 64),
            nn.ReLU(),
            nn.Linear(64, 7)
        )

    def forward(self, x):
        return self.network(x)
      
    def save(self,path):
      torch.save(self.network.state_dict(), path)
    
    def load(self, path):
      self.network.load_state_dict(torch.load(path))

    def train_net(self,epochs, lr, train_loader, test_loader,opt_func, loss_func):
      print(self.device)
      self.network.train()

      optimizer = opt_func(self.network.parameters(), lr=lr)

      for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
          inputs, labels = data
          inputs, labels = inputs.to(self.device),  labels.to(self.device) # move to GPU is possible
          
          # zero the parameter gradients
          optimizer.zero_grad()

          # forward + backward + optimize
          outputs = self.network(inputs)
          loss = loss_func(outputs, labels)
          loss.backward()
          optimizer.step()

          # print statistics
          running_loss += loss.item()
          if i % 2000 == 1999:    # print every 2000 mini-batches
              print(f'[{epoch + 1}, {i + 1:5d}] train loss: {running_loss / 2000:.3f}')
              running_loss = 0.0

      print('Finished Training')
    
    @torch.no_grad()
    def test(self,test_loader):
      self.network.eval()
      correct = 0
      total = 0
      for data in test_loader:
        inputs, labels = data
        inputs, labels = inputs.to(self.device),  labels.to(self.device) 

        predictions = self.network(inputs)
        _, predicted = torch.max(predictions.data, 1)

        total += len(data)
        correct += torch.sum(predicted == labels).item()
      
      return correct/total

In [ ]:
# Run K-Folds (Vanilla CNN w/Augmentations)
csv_file = "./dataverse_files/HAM10000_metadata.csv"
directory = "./dataverse_files"

transform = transforms.Compose(
      [
        transforms.ToTensor(),
        transforms.RandomCrop(224),
        transforms.RandomRotation(degrees=(13)),
        transforms.ColorJitter(brightness=.5, hue=.3),
        transforms.Normalize((0.1411, 0.0923, 0.5270), (0.3407, 0.3058, 0.2824))
          
      ]
    )

batch_size = 32
datasetname = "HAM10000_images_off"
dataset = Ham10000(csv_file, directory, transform, datasetname)

model = Net()


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

kfold(model, dataset, device)

In [ ]:
# Process Results

fold1trainacc = [66.67498751872192, 66.96205691462806, 66.96205691462806, 66.89965052421368, 66.96205691462806, 66.84972541188218, 66.96205691462806, 66.9370943584623, 66.96205691462806, 66.96205691462806]
fold1testacc = [66.89965052421368, 66.89965052421368, 66.89965052421368, 66.89965052421368, 66.89965052421368, 66.89965052421368, 66.89965052421368, 66.89965052421368, 66.89965052421368, 66.89965052421368]
total1_train_loss =  [587.4317564666271, 555.7362589538097, 533.5898955762386, 531.2370494306087, 526.737876355648, 535.5994701683521, 526.8527587652206, 522.9258488416672, 517.0247338712215, 515.8226729631424]

fold2trainacc = [66.8372441337993, 67.01198202695956, 67.01198202695956, 67.01198202695956, 67.01198202695956, 66.40039940089865, 67.01198202695956, 67.01198202695956, 67.01198202695956, 67.01198202695956]
fold2testacc = [66.69995007488767, 66.69995007488767, 66.69995007488767, 66.69995007488767, 66.69995007488767, 66.69995007488767, 66.69995007488767, 66.69995007488767, 66.69995007488767, 66.69995007488767]
total2_train_loss= [577.734813272953, 572.6809532642365, 571.536487609148, 569.8435501754284, 571.6515056490898, 618.4085868299007, 571.6554693877697, 570.0561816990376, 569.7348641455173, 568.4866904914379]

fold3trainacc = [66.60009985022467, 66.88716924613081, 66.74987518721917, 66.7373939091363, 66.81228157763356, 66.89965052421368, 66.89965052421368, 66.89965052421368, 66.89965052421368, 66.89965052421368]
fold3testacc = [67.1492760858712, 67.1492760858712, 67.1492760858712, 67.1492760858712, 67.1492760858712, 67.1492760858712, 67.1492760858712, 67.1492760858712, 67.1492760858712, 67.1492760858712]
total3_train_loss = [569.2010554969311, 539.6914494037628, 549.1378348469734, 544.0546608269215, 585.012408643961, 570.9453338682652, 570.7000705003738, 569.9297728538513, 569.4372866749763, 569.206324338913]

fold4trainacc = [66.41288067898152, 66.56265601597603, 66.56265601597603, 66.45032451323016, 66.51273090364454, 66.5751372940589, 66.56265601597603, 66.58761857214179, 66.5751372940589, 66.55017473789316]
fold4testacc = [68.44732900649026, 68.44732900649026, 68.44732900649026, 68.44732900649026, 68.44732900649026, 68.44732900649026, 68.44732900649026, 68.44732900649026, 68.44732900649026, 68.44732900649026]
total4_train_loss= [567.0311739295721, 541.0548948049545, 538.174273788929, 531.5657752454281, 546.1777322292328, 521.7583494782448, 519.0866031050682, 523.7675650566816, 510.0987524539232, 513.779454022646]

fold5trainacc = [67.09935097353969, 67.2990514228657, 67.13679480778832, 67.26160758861707, 67.2990514228657, 67.16175736395407, 67.2990514228657, 67.2990514228657, 67.23664503245132, 67.27408886669996]
fold5testacc = [65.5516724912631, 65.5516724912631, 65.5516724912631, 65.5516724912631, 65.5516724912631, 65.5516724912631, 65.5516724912631, 65.5516724912631, 65.5516724912631, 65.5516724912631]
total5_train_loss=  [572.5991007685661, 534.6310192644596, 543.1988770961761, 553.1111596822739, 519.6321686804295, 567.483310252428, 532.9011108875275, 522.2456321418285, 539.5499770641327, 567.5038605332375]


total_train_loss_append = [total1_train_loss, total2_train_loss, total3_train_loss, total4_train_loss, total5_train_loss]
total_train_loss = []
for i in range(len(total1_train_loss)):
  total_loss = []
  for j in range(len(total_train_loss_append)):
    total_loss.append(total_train_loss_append[j][i])
  total_train_loss.append(sum(total_loss) / len(total_loss))
print(total_train_loss)


total_train_acc_append = [fold1trainacc, fold2trainacc, fold3trainacc, fold4trainacc, fold5trainacc]

total_train_acc = []
for i in range(len(total1_train_loss)):
  total_acc = []
  for j in range(len(total_train_acc_append)):
    total_acc.append(total_train_acc_append[j][i])
  total_train_acc.append(sum(total_acc) / len(total_acc))
print(total_train_acc)


total_test_acc_append = [fold1testacc, fold2testacc, fold3testacc, fold4testacc, fold5testacc]

total_test_acc = []
for i in range(len(total1_train_loss)):
  total_acc = []
  for j in range(len(total_test_acc_append)):
    total_acc.append(total_test_acc_append[j][i])
  total_test_acc.append(sum(total_acc) / len(total_acc))
print(total_test_acc)

In [ ]:
# Loss Graph
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import seaborn as sns
import pandas as pd

epochs = list(range(0,len(total_train_loss)))

data = pd.DataFrame({
    'Epoch': epochs,
    'Loss': total_train_loss,
})

sns.set_theme(style="darkgrid")
sns.lineplot(x='Epoch',y='Loss',data=data).set(title="Train Loss")

In [ ]:
# Accuracy Graph
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

epochs = list(range(0,len(total_train_acc)))

data = pd.DataFrame({
    'Epoch': epochs,
    'Training Accuracy': total_train_acc,
    'Testing Accuracy': total_test_acc
})

sns.set_theme(style="darkgrid")
sns.lineplot(x='Epoch',y='Accuracy',hue='variable',data=pd.melt(data, ['Epoch'],value_name="Accuracy")).set(title="Train vs Test Accuracy")
